CAMS

- mean, median, std, min, max for PM2.5 values by year and year-month

- maps for both results


Yearly

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from collections import defaultdict

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_singlvl\5compile_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store yearly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 

    # Extract the year from the 'time' dimension
    year = ds['time'].dt.year.values[0]  # Extract the year from the first time step
    
    # Get the total pm2p5 data as a numpy array
    pm2p5_data = ds['pm2p5'].values  # Numpy array (all time steps)

    # Compute statistics using numpy's nan functions
    mean = np.nanmean(pm2p5_data, axis=0)
    median = np.nanmedian(pm2p5_data, axis=0)
    std = np.nanstd(pm2p5_data, axis=0)
    max_ = np.nanmax(pm2p5_data, axis=0)
    min_ = np.nanmin(pm2p5_data, axis=0)
    total = np.nansum(pm2p5_data, axis=0)
    
    # Combine results into a new dataset
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = year  # Add the year directly from time
    stats_df = stats_df.set_index(['Year', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all years into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)


                              Mean     Median        Std         Max  Min  \
Year latitude longitude                                                     
2003 66.0     -12.00     10.771490   6.846619  11.513706   92.672036  0.0   
              -11.25     10.884861   7.004896  11.549816   88.938802  0.0   
              -10.50     10.919763   7.226987  11.420478   81.252733  0.0   
              -9.75      10.984016   7.186605  11.417963   88.039358  0.0   
              -9.00      11.016679   7.239998  11.346276   93.218144  0.0   
...                            ...        ...        ...         ...  ...   
2024 34.5      33.00     10.772711   7.979452  10.361039   96.834923  0.0   
               33.75     10.846634   7.856897  10.658591   95.575157  0.0   
               34.50     11.186935   7.908682  11.155924   79.847382  0.0   
               35.25     16.197813  11.658756  15.055274  109.020218  0.0   
               36.00     33.452877  25.091537  24.362345  146.947968  0.0   

In [2]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_singlvl\yearly_pm2p5_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


Dataset saved to: E:\IPMA\CAMS\chem_singlvl\yearly_pm2p5_stats.nc


In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_singlvl\yearly_pm2p5_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_singlvl\maps_year\pm2p5"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Projection (assume data is in PlateCarree, common for lat/lon grids)
data_crs = ccrs.PlateCarree()

# Loop through years
for year in ds['Year'].values:
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw={'projection': ccrs.PlateCarree()})

    # Select data for current year
    data = ds[var].sel(Year=year)

    # Plot the data
    im = ax.pcolormesh(ds['longitude'], ds['latitude'], data,
                       cmap=cmap, vmin=vmin, vmax=vmax,
                       shading='auto', transform=data_crs)

    # Add features (land, ocean, borders)
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.set_extent([-12, 36, 34, 66], crs=data_crs)  # Adjust to the working area

    # Gridlines (bottom and left only)
    gl = ax.gridlines(draw_labels=True, linewidth=0)
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True

    # Title and colorbar
    ax.set_title(f'{title} - {year}', fontsize=12)

    # Create a colorbar with more space
    cbar = fig.colorbar(im, ax=ax, orientation='horizontal', label='µg/m³', shrink=0.7, pad=0.1)
    
    # Adjust the colorbar's position
    cbar.ax.yaxis.set_ticks_position('right')  # Put colorbar ticks on the right side

    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"mean_concentration_pm2p5_{year}.png"), dpi=150)
    plt.close(fig)

print("✅ Mean temperature maps with adjusted colorbar spacing generated!")


✅ Mean temperature maps with adjusted colorbar spacing generated!


Monthly

In [4]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_singlvl\4conversion_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store monthly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 
    
    # Extract year and month from the 'time' dimension
    ds.coords['year'] = ds['time'].dt.year
    ds.coords['month'] = ds['time'].dt.month
    
    # Get the total pm2p5 data as a numpy array (time, latitude, longitude)
    pm2p5_data = ds['pm2p5'].values
    
    # Calculate statistics using numpy's nan functions along the 'time' dimension
    mean = np.nanmean(pm2p5_data, axis=0)
    median = np.nanmedian(pm2p5_data, axis=0)
    std = np.nanstd(pm2p5_data, axis=0)
    max_ = np.nanmax(pm2p5_data, axis=0)
    min_ = np.nanmin(pm2p5_data, axis=0)
    total = np.nansum(pm2p5_data, axis=0)
    
    # Create a new xarray Dataset for the statistics
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})
    
    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = ds['year'].values[0]  # Add the year directly from time
    stats_df['Month'] = ds['month'].values[0]  # Add the month directly from time
    stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all months into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)
df_final


                                    Mean     Median        Std         Max  \
Year Month latitude longitude                                                
2003 1     66.0     -12.00      9.928696   6.394427  10.858686   58.397294   
                    -11.25     10.208592   6.432947  11.301746   63.790184   
                    -10.50     10.525144   6.933716  11.743281   68.874909   
                    -9.75      10.809389   7.164839  11.920285   70.723899   
                    -9.00      11.107303   7.473005  12.074469   75.192294   
...                                  ...        ...        ...         ...   
2024 11    34.5      33.00      8.420444   5.480729   7.517126   37.809628   
                     33.75     10.194614   6.069456   9.711091   51.007420   
                     34.50     11.617288   6.765959  11.881586   58.955494   
                     35.25     15.496955  10.641173  13.907671   62.911114   
                     36.00     38.358418  33.683392  23.636988  

Mean     Median        Std         Max  \
Year Month latitude longitude                                                
2003 1     66.0     -12.00      9.928696   6.394427  10.858686   58.397294   
                    -11.25     10.208592   6.432947  11.301746   63.790184   
                    -10.50     10.525144   6.933716  11.743281   68.874909   
                    -9.75      10.809389   7.164839  11.920285   70.723899   
                    -9.00      11.107303   7.473005  12.074469   75.192294   
...                                  ...        ...        ...         ...   
2024 11    34.5      33.00      8.420444   5.480729   7.517126   37.809628   
                     33.75     10.194614   6.069456   9.711091   51.007420   
                     34.50     11.617288   6.765959  11.881586   58.955494   
                     35.25     15.496955  10.641173  13.907671   62.911114   
                     36.00     38.358418  33.683392  23.636988  102.818031   

                                    Min        Total  
Year Month latitude longitude                         
2003 1     66.0     -12.00     0.231124  2462.316503  
                    -11.25     0.385206  2531.730701  
                    -10.50     0.539289  2610.235770  
                    -9.75      0.693372  2680.728545  
                    -9.00      0.770413  2754.611138  
...                                 ...          ...  
2024 11    34.5      33.00     0.177351  2020.906494  
                     33.75     0.104592  2446.707520  
                     34.50     0.001819  2788.148926  
                     35.25     0.295586  3719.269287  
                     36.00     3.958519  9206.020508  

[735085 rows x 6 columns]

In [6]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_singlvl\monthly_pm2p5_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


Dataset saved to: E:\IPMA\CAMS\chem_singlvl\monthly_pm2p5_stats.nc


In [7]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_singlvl\monthly_pm2p5_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_singlvl\maps_month\pm2p5"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years and months
for year in ds['Year'].values:
    for month in ds['Month'].values:
        data = ds[var].sel(Year=year, Month=month)

        # Skip empty or all-NaN values
        if np.isnan(data).all():
            continue

        # Plotting
        fig = plt.figure(figsize=(7, 6))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'{title} - {year} / {month:02d}', fontsize=12)
        ax.coastlines(resolution='10m', linewidth=0.8)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = True
        gl.left_labels = True

        # Colorbar
        cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.7, pad=0.1)
        cbar.set_label('µg/m³')
        cbar.ax.tick_params(labelsize=8)

        # Save figure
        plt.tight_layout()
        filename = f"mean_concentration_pm2p5_{year}{month:02d}.png"
        plt.savefig(os.path.join(output_folder, filename), dpi=150)
        plt.close(fig)

print("✅ Monthly temperature maps generated for all years and months!")


✅ Monthly temperature maps generated for all years and months!


In [8]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_singlvl\monthly_pm2p5_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_singlvl\maps_month\pm2p5"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years
for year in ds['Year'].values:
    fig, axs = plt.subplots(3, 4, figsize=(15, 10),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    fig.suptitle(f'{title} - {year}', fontsize=16)

    for i, month in enumerate(ds['Month'].values):
        row, col = divmod(i, 4)
        ax = axs[row, col]

        # Extract data
        data = ds[var].sel(Year=year, Month=month)

        # Skip if no data
        if np.isnan(data).all():
            ax.set_visible(False)
            continue

        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'Month {month:02d}', fontsize=10)
        ax.coastlines(resolution='10m', linewidth=0.6)
        ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = row == 2
        gl.left_labels = col == 0

    # Add a single colorbar for all subplots
    cbar_ax = fig.add_axes([0.92, 0.25, 0.015, 0.5])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label('µg/m³')

    # Save figure
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    filename = f"mean_concentration_pm2p5_year_{year}.png"
    plt.savefig(os.path.join(output_folder, filename), dpi=150)
    plt.close(fig)

print("✅ Yearly temperature grids generated (3x4 months per year)!")


C:\Users\sofia\AppData\Local\Temp\ipykernel_9124\2406401250.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_9124\2406401250.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_9124\2406401250.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_9124\2406401250.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_9124\2406401250.py:72: UserWarning: This figure includes Axes th

✅ Yearly temperature grids generated (3x4 months per year)!
